In [9]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Amdahl's Law

## Optimization is a moving target

## Amdahl's Law -- optimizating is a moving target

In [11]:
render_code("./gpusort/main.cu", lang="c++", show="bitonic_sort")

// ./gpusort/main.cu:63-92 (30 lines)
void bitonic_sort(float *values, float *out_values, int num_of_elements)
{
  float *dev_values;
  size_t size = num_of_elements * sizeof(float);
  int BLOCKS = num_of_elements/THREADS_PER_BLOCK; 

  cudaMalloc((void**) &dev_values, size);
  sdkStartTimer(&uploadTimer);
  cudaMemcpy(dev_values, values, size, cudaMemcpyHostToDevice);
  sdkStopTimer(&uploadTimer); 
  dim3 blocks(BLOCKS,1);    /* Number of blocks   */
  dim3 threads(THREADS_PER_BLOCK,1);  /* Number of threads  */

  sdkStartTimer(&totalTimer); 
  int j, k;
  /* Major step */
  for (k = 2; k <= num_of_elements; k <<= 1) {
    /* Minor step */
    for (j=k>>1; j>0; j=j>>1) {
      bitonic_sort_step<<<blocks, threads>>>(dev_values, j, k);
    }
  }
		cudaThreadSynchronize(); 
  sdkStopTimer(&totalTimer); 
  sdkStartTimer(&downloadTimer);
  cudaMemcpy((void *)out_values, (void *)dev_values,  num_of_elements*sizeof(float), cudaMemcpyDeviceToHost);
//		checkCudaErrors(cudaMemcpy((void *)out_values, (void *)dev_values, num_of_elements*sizeof(float), cudaMemcpyDeviceToHost));
  sdkStopTimer(&downloadTimer); 
  cudaFree(dev_values);
}

In [3]:
! nvidia-smi

Thu Aug  7 12:44:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    Off |   00000000:01:00.0 Off |                  N/A |
| 32%   33C    P8             12W /  100W |      15MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
! cd gpusort; make clean; make
# ! ssh htseng@azelf "source ./courses/CS203/demo/amdahlslaw/gpusort/run_CPU"
! echo "ET, FileInput, CPU_Kernel, GPU_Kernel, Host2GPU, GPU2Host" > ./gpusort/sort.csv
! /usr/sbin/clearcache > /dev/null
! echo "File on H.D.D.; Sorting on CPU"
! cd gpusort; source ./run_CPU 2>> sort.csv
display_df_mono(render_csv("./gpusort/sort.csv"))

rm -f	*.o hybridsort hybridsort_cpu bitonic
/usr/local/cuda/bin/nvcc -DTIMER -O3 -w -g    -DCPU -DHAVE_LINUX_PERF_EVENT_H -DREADING_FROM_BINARY         main.cu -o hybridsort_cpu
/usr/local/cuda/bin/nvcc -DTIMER -O3 -w -g    -DHAVE_LINUX_PERF_EVENT_H -DREADING_FROM_BINARY         main.cu -o hybridsort
File on H.D.D.; Sorting on CPU
Sorting list of 134217728 floats
FileInput 1.834462 seconds
Sorting on CPU...done.
Total CPU execution time: 15.383359 seconds


,index,ET,FileInput,CPU_Kernel,GPU_Kernel,Host2GPU,GPU2Host
0,0,17.218188,1.834462,15.383359,0.000000,0.000000,0.000000


In [13]:
! /usr/sbin/clearcache > /dev/null
! echo "File on H.D.D.; Sorting on GPU"
! cd gpusort; source ./run 2>> sort.csv

File on H.D.D.; Sorting on GPU
Sorting list of 134217728 floats
FileInput 1.828547 seconds
Sorting on GPU...GPU iterations: 1
Total GPU Sort execution time: 0.910748 seconds
    - Upload		: 0.037590 seconds
    - Download		: 0.158556 seconds


In [14]:
display_df_mono(render_csv("./gpusort/sort.csv"))

,index,ET,FileInput,CPU_Kernel,GPU_Kernel,Host2GPU,GPU2Host
0,0,17.218188,1.834462,15.383359,0.000000,0.000000,0.000000
1,1,3.101171,1.828547,0.000000,0.910748,0.037590,0.158556


In [7]:
! cp ~/courses/CSE142/demo/300000000.bin /tmp
! /usr/sbin/clearcache > /dev/null
! echo "File on S.S.D.; Sorting on GPU"
! cd gpusort; source ./run_SSD 2>> sort.csv

File on S.S.D.; Sorting on GPU
Sorting list of 134217728 floats
FileInput 0.329169 seconds
Sorting on GPU...GPU iterations: 1
Total GPU Sort execution time: 0.926812 seconds
    - Upload		: 0.037186 seconds
    - Download		: 0.155048 seconds


In [8]:
display_df_mono(render_csv("./gpusort/sort.csv"))

,index,ET,FileInput,CPU_Kernel,GPU_Kernel,Host2GPU,GPU2Host
0,0,17.408289,1.828791,15.579085,0.000000,0.000000,0.000000
1,1,3.109873,1.828919,0.000000,0.922205,0.037119,0.162884
2,2,1.581998,0.329169,0.000000,0.926812,0.037186,0.155048
